**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



**Set global variable**

In [4]:
## init: region folder
vec = c("fcc_astarr_macs_input_overlap", "fcc_astarr_macs_input_union")
names(vec) = vec

VEC_TXT_FOLDER = vec
for (txt in vec) {cat(txt, "\n")}

fcc_astarr_macs_input_overlap 
fcc_astarr_macs_input_union 


In [5]:
vec = c("CRISPRi-HCRFF", "CRISPRi-Growth", "ENCODE-rE2G Benchmark")
names(vec) = vec
VEC_TXT_ASSAY_OUT = vec

vec = c("ASTARR", "WSTARR", "LMPRA", "TMPRA", "ATAC")
names(vec) = vec
VEC_TXT_ASSAY_INP = vec

**Check input files**

In [6]:
txt_folder = "fcc_astarr_macs_input_union"
txt_fdiry  = file.path(FD_RES, "analysis_fcc_correlation", txt_folder)
txt_fname  = "*.rds"
txt_fglob  = file.path(txt_fdiry, txt_fname)

vec = Sys.glob(txt_fglob)
for (txt in vec) {cat(txt, "\n")}

/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/analysis_fcc_correlation/fcc_astarr_macs_input_union/data.region.merge.group_crispr.score_starrmpra.rds 


## Import data

In [12]:
### set file directory
txt_foldr = "fcc_astarr_macs_input_union"
txt_fdiry = file.path(FD_RES, "analysis_fcc_correlation", txt_foldr)
txt_fname = "data.region.merge.group_crispr.score_starrmpra.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

### read data
lst = readRDS(txt_fpath)

### assign and show
lst_dat_region_merge = lst

res = lapply(lst, dim)
print(res)

dat = lst[[1]]
fun_display_table(head(dat, 3))

$`0.01`
[1] 610756      7

$`0.02`
[1] 610756      7

$`0.03`
[1] 610756      7

$`0.04`
[1] 610756      7

$`0.05`
[1] 610756      7

$`0.06`
[1] 610756      7

$`0.07`
[1] 610756      7

$`0.08`
[1] 610756      7

$`0.09`
[1] 610756      7

$`0.10`
[1] 610756      7

$`0.20`
[1] 610756      7

$`0.30`
[1] 610756      7

$`0.40`
[1] 610756      7

$`0.50`
[1] 610756      7



CutOff_CRISPR,Region,Assay_CRISPR,Group_CRISPR,Score_CRISPR,Assay_STARRMPRA,Score_STARRMPRA
0.01,chr10:100009096-100010466,CRISPR,0,-0.0378617,ASTARR,-1.2216566
0.01,chr10:100009096-100010466,CRISPR,0,-0.0378617,ATAC,2.3214952
0.01,chr10:100009096-100010466,CRISPR,0,-0.0378617,WSTARR,0.5896695


## Helper function

In [13]:
fun_auroc = function(vec_boo_group, vec_num_score) {
    ### sort the group by score
    idx = order(vec_num_score)
    vec = vec_boo_group[idx]
    vec_group = vec

    ### calculate total positive and negative cases
    num_total_pos = sum( vec_group)
    num_total_neg = sum(!vec_group)
    if (num_total_pos == 0 || num_total_neg == 0) {
        stop("Need at least one TRUE and one FALSE")
    }
    
    ### calculate positive and negative rates
    vec_num_tpr = (num_total_pos - cumsum( vec_group)) / num_total_pos
    vec_num_fpr = (num_total_neg - cumsum(!vec_group)) / num_total_neg

    ### convert to sensitivity and specificity
    vec_num_sens = vec_num_tpr
    vec_num_spec = 1 - vec_num_fpr
    
    ### estimate the area under curve
    num_width = diff(c(0, 1 - vec_num_sens))
    num_auc   = sum(vec_num_spec * num_width)

    ### combine and return the results
    lst = list(
        "Sensitivity" = vec_num_sens,
        "Specificity" = vec_num_spec,
        "AUROC"       = num_auc
    )
    return(lst)
}

## Perform logistic regression for each assay pair

**Show one table**

In [9]:
lst = lst_dat_region_merge
dat = lst[[1]]

dat_region_merge = dat
fun_display_table(head(dat))

CutOff_CRISPR,Region,Assay_CRISPR,Group_CRISPR,Score_CRISPR,Assay_STARRMPRA,Score_STARRMPRA
0.01,chr10:100009096-100010466,CRISPR,0,-0.0378617,ASTARR,-1.2216566
0.01,chr10:100009096-100010466,CRISPR,0,-0.0378617,ATAC,2.3214952
0.01,chr10:100009096-100010466,CRISPR,0,-0.0378617,WSTARR,0.5896695
0.01,chr10:100009096-100010466,CRISPRi-Growth,0,-0.0378617,ASTARR,-1.2216566
0.01,chr10:100009096-100010466,CRISPRi-Growth,0,-0.0378617,ATAC,2.3214952
0.01,chr10:100009096-100010466,CRISPRi-Growth,0,-0.0378617,WSTARR,0.5896695


**Run logit: CRISPR~STARR/MPRA/ATAC and calculate ROC curve**

In [14]:
lst = lst_dat_region_merge
lst = lapply(lst, function(dat_region_merge){
    lst = lapply(VEC_TXT_ASSAY_OUT, function(txt_assay_out){
        lst = lapply(VEC_TXT_ASSAY_INP, function(txt_assay_inp){
            ### get group/score
            dat = dat_region_merge
            dat = dat %>% 
                dplyr::filter(
                    Assay_CRISPR    == txt_assay_out, 
                    Assay_STARRMPRA == txt_assay_inp
                )
    
            ### run logistic regression
            fit = glm(Group_CRISPR ~ Score_STARRMPRA, data = dat, family = "binomial")
            vec_boo_group = (dat$Group_CRISPR == 1)
            vec_num_score = predict(fit, type = "response")
            
            ### calculate assay performance
            lst = fun_auroc(vec_boo_group, vec_num_score)
            vec_num_spec = lst[["Specificity"]]
            vec_num_sens = lst[["Sensitivity"]]
            num_auroc    = lst[["AUROC"]]
        
            ### summarize the results
            txt_title = paste0("(", round(num_auroc, 2), ")")
            txt_title = paste(txt_assay_inp, txt_title)
        
            dat = data.frame(
                #"Assay_CRISPR"    = txt_assay_out,
                #"Assay_STARRMPRA" = txt_assay_inp,
                "x"     = 1 - vec_num_spec,
                "y"     = vec_num_sens,
                "AUROC" = num_auroc,
                "Performance" = txt_title
            )
            return(dat)
        })
    
        ### concat results across different STARRMPRA assay
        dat = bind_rows(lst, .id = "Assay_STARRMPRA")
        return(dat)
    })

    ### concat results across different CRISPR assay
    dat = bind_rows(lst, .id = "Assay_CRISPR")
    return(dat)
})

### concat results across different hit rate
dat = bind_rows(lst, .id = "CutOff_CRISPR")

### assign and show
dat_logit_roc = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 4374832       7


CutOff_CRISPR,Assay_CRISPR,Assay_STARRMPRA,x,y,AUROC,Performance
0.01,CRISPRi-HCRFF,ASTARR,0.9992401,1,0.6991424,ASTARR (0.7)
0.01,CRISPRi-HCRFF,ASTARR,0.9984802,1,0.6991424,ASTARR (0.7)
0.01,CRISPRi-HCRFF,ASTARR,0.9977204,1,0.6991424,ASTARR (0.7)


## Export results

In [15]:
### set file directory
txt_foldr = "fcc_astarr_macs_input_union"
txt_fdiry = file.path(FD_RES, "analysis_fcc_correlation", txt_foldr)
txt_fname = "result.cripsr_vs_starrmpra.logit.roc.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

### write table
dat = dat_logit_roc
write_tsv(dat, txt_fpath)